In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

Consultas

In [2]:
#@app.get('/get_max_duration/{anio}/{plataforma}/{dtype}')
def get_max_duration(anio: int, plataforma: str, dtype: str):
    df = pd.read_csv("platforms_and_ratings.csv")

    # Filtrar los datos para incluir sólo películas
    df_movies = df[df['type'] == 'movie']
    
    # Filtrar los datos para incluir sólo películas que correspondan al año y la plataforma especificados
    df_movies = df_movies[(df_movies['release_year'] == anio) & (df_movies['platform'] == plataforma)]
    
    # Filtrar los datos para incluir sólo películas con el tipo de duración especificado
    df_movies = df_movies[df_movies['duration_type'] == dtype]
    
    # Ordenar los datos por duración en orden descendente
    df_movies = df_movies.sort_values(by='duration_int', ascending=False)
    
    # Primera fila del dataframe resultante, que tendrá la película con la mayor duración
    max_duration_movie = df_movies.iloc[0]['title']
    
    return {'pelicula': max_duration_movie}

In [3]:
get_max_duration(2011,'amazon','min')

{'pelicula': 'kanchana'}

In [4]:
#@app.get('/get_score_count/{plataforma}/{scored}/{anio}')
def get_score_count(plataforma: str, scored: float, anio: int):
    df = pd.read_csv("platforms_and_ratings.csv")

    # Filtrar los datos para incluir sólo películas
    df_movies = df[df['type'] == 'movie']
    
    # Filtrar los datos para incluir sólo películas que correspondan al año y la plataforma especificados
    df_movies = df_movies[(df_movies['release_year'] == anio) & (df_movies['platform'] == plataforma)]
    
    # Filtrar los datos para incluir sólo películas con un puntaje mayor que el puntaje especificado
    df_movies = df_movies[df_movies['score_mean'] > scored]
    
    # Contar el número de filas en el dataframe 
    count = len(df_movies)
    
    return {
        'plataforma': plataforma,
        'cantidad': count,
        'anio': anio,
        'score': scored
    }

In [5]:
get_score_count('disney',1.4,2012)

{'plataforma': 'disney', 'cantidad': 29, 'anio': 2012, 'score': 1.4}

In [6]:
#@app.get('/get_count_platform/{plataforma}')
def get_count_platform(plataforma: str):
    df = pd.read_csv("platforms_and_ratings.csv")

    # Filtrar los datos para incluir sólo películas
    df_movies = df[df['type'] == 'movie']
    
    # Filtrar los datos para incluir sólo películas que correspondan a la plataforma especificada
    if plataforma == 'amazon':
        df_movies = df_movies[df_movies['platform'] == 'amazon']
    elif plataforma == 'netflix':
        df_movies = df_movies[df_movies['platform'] == 'netflix']
    elif plataforma == 'hulu':
        df_movies = df_movies[df_movies['platform'] == 'hulu']
    elif plataforma == 'disney':
        df_movies = df_movies[df_movies['platform'] == 'disney']
    else:
        return {'error': 'Plataforma no reconocida'}
    
    # Contar el número de filas en el dataframe 
    count = len(df_movies)
    
    return {'plataforma': plataforma, 'peliculas': count}

In [7]:
get_count_platform('hulu')

{'plataforma': 'hulu', 'peliculas': 1484}

In [8]:
#@app.get('/get_actor/{plataforma}/{anio}')
def get_actor(plataforma: str, anio: int):
    df = pd.read_csv("platforms_and_ratings.csv")

   # Filtrar por plataforma y año
    df_filtered = df[(df['platform'] == plataforma) & (df['release_year'] == anio)]

    # Dividir la columna cast en listas de actores, y manejar NaN
    df_filtered.loc[:, 'cast'] = df_filtered['cast'].apply(lambda x: [] if pd.isna(x) else x.split(', '))

    # Crear una lista con todos los actores en el DataFrame
    all_actors = [actor for cast_list in df_filtered['cast'] for actor in cast_list]

    # Obtener el actor que más se repite
    actor_counts = pd.Series(all_actors).value_counts()
    most_common_actor = actor_counts.index[0]
    most_common_actor_appearances = actor_counts[0]

    return {
        'plataforma': plataforma,
        'anio': anio,
        'actor': most_common_actor,
        'apariciones': most_common_actor_appearances
    }


In [9]:
get_actor('netflix',2014)

C:\Users\Usuario\AppData\Local\Temp\ipykernel_7376\2247508944.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered.loc[:, 'cast'] = df_filtered['cast'].apply(lambda x: [] if pd.isna(x) else x.split(', '))


{'plataforma': 'netflix',
 'anio': 2014,
 'actor': 'miyuki sawashiro',
 'apariciones': 5}

In [10]:
#@app.get('/prod_per_county/{tipo}/{pais}/{anio}')
def prod_per_county(tipo: str, pais: str, anio: int):
    df = pd.read_csv("platforms_and_ratings.csv")

    # Filtrar por tipo de contenido, país y año
    df_filt = df[(df['type'] == tipo) & (df['country'] == pais) & (df['release_year'] == anio)]

    # Agrupar por país, año y tipo de contenido
    grouped = df_filt.groupby(['country', 'release_year', 'type'])

    # Contar el número de películas o series por grupo
    count = grouped['title'].count().reset_index(name='count')

    # Obtener el número de películas o series según el tipo especificado
    result = count[count['type'] == tipo]['count'].values[0]
    
    return {'pais': pais, 'anio': anio, tipo: result}


In [11]:
prod_per_county('tv show','france',2018)

{'pais': 'france', 'anio': 2018, 'tv show': 12}

In [12]:
#@app.get('/get_contents/{rating}')
def get_contents(rating: str):
    df = pd.read_csv("platforms_and_ratings.csv")

    # Cuenta el número de filas donde el valor de la columna "rating" es igual a la entrada "rating"
    count = len(df[df['rating'] == rating])
    
    return {'rating': rating, 'contenido': count}

In [13]:
get_contents('18+')

{'rating': '18+', 'contenido': 1243}

Sistema de recomendación

In [14]:
#@app.get('/get_recomendation/{title}')
def get_recomendation(title:str):
    df = pd.read_csv("platforms_and_ratings.csv")

    # Crear una instancia del vectorizador TF-IDF
    vectorizer = CountVectorizer(stop_words="english")

    # Obtener la matriz de términos de los títulos
    title_matrix = vectorizer.fit_transform(df["title"])

    # Calcular la similitud coseno entre todos los títulos
    cosine_similarities = cosine_similarity(title_matrix)

    # Obtener el índice de la película de interés
    idx = df.index[df['title'] == title].tolist()[0]

    # Obtener las puntuaciones de similitud de esa película con todas las películas
    similarity_scores = list(enumerate(cosine_similarities[idx]))

    # Ordenar las películas según puntuación de similitud
    similarity_scores = sorted(similarity_scores, key=lambda x: x[1], reverse=True)

    # Obtener los índices de las películas recomendadas
    movie_indices = [i[0] for i in similarity_scores[1:11]]

    # Recomendaciones
    recommendations = df.loc[movie_indices, "title"].tolist()[:5]
    return {'recomendación': recommendations}

In [15]:
df = pd.read_csv("platforms_and_ratings.csv")
# Observar algunos títulos
df.title[0:30]

0                        ricky velez: here's everything
1                                          silent night
2                                          the marksman
3                                                  gaia
4                                              settlers
5                         the halloween candy magic pet
6                                    the evil next door
7                                the next thing you eat
8                                                queens
9                                      the bachelorette
10    the real queens of hip-hop: the women who chan...
11                                          dream horse
12    out of the shadows: the man behind the steele ...
13                                            wakefield
14                                      home sweet home
15    showtime championship boxing: lopez vs. salido...
16      showtime championship boxing: wilder vs. molina
17                            america's book of 

In [16]:
#Probando el sistema
get_recomendation('silent night')

{'recomendación': ['the first silent night',
  'the night',
  'i am the night',
  'all night',
  'one night']}

In [17]:
#Probando el sistema
get_recomendation('the marksman')

{'recomendación': ['the marksman',
  'lord marksman and vanadis',
  "ricky velez: here's everything",
  'silent night',
  'gaia']}

In [18]:
#Probando el sistema
get_recomendation('dream horse')

{'recomendación': ['get a horse!',
  'can you dream of more',
  'horse sense',
  "red's dream",
  'my dream farm']}

In [19]:
#Probando el sistema
get_recomendation('sleepwalker')

{'recomendación': ['sleepwalker',
  "ricky velez: here's everything",
  'silent night',
  'the marksman',
  'gaia']}

In [20]:
#Probando el sistema
get_recomendation('cheer camp killer')

{'recomendación': ['camp nowhere',
  'killer among us',
  'cheer up',
  'i am a killer',
  'cheer']}